---
title: "Testing regular expressions for equivalence"
author: "Martin Ong"
date: "2024-1-12"
format: 
  html:
    toc: true
    embed-resources: true
    code-fold: false
    code-tools: true
    code-line-numbers: true
    number-sections: true
---

# Equivalence
Any two languages (sets of strings) are equivalent if they each contain
the same strings. Often, when working with regular languages, we deal with
objects which recognise a particular language, however it is not immediately
clear from looking at such an object what the language it recognises actually
is.

A common problem when dealing with such objects is determining whether two
different objects are equivalent, in the sense that they recognise the same
language.

In this post, we will focus on finite automata. <!-- TODO: reword -->

In [ ]:
class Automaton(Protocol, Generic[T, S]):
    """
    T: type of a state
    S: type of a state's transition table
    """
    start: T
    states: set[T]
    final: set[T]
    transitions: dict[T, S]

    def add_state(self, transitions: S) -> None:
        ...

    def make_final(self, state: T) -> None:
        assert state in self.states
        self.final.add(state)

    def update(self, state: T, transitions: S) -> None:
        self.transitions[state] = transitions

    def evaluate(self, s: str) -> bool:
        ...

# Deterministic Finite Automata
Deteriministic finite automata (DFAs) are one such object which behave nicely.
It is well known that every DFA recognises a regular language, and every regular
language is recognised by some DFA. Thus, we will begin by trying to compare
two DFAs for equivalence.

We can represent a DFA with the following python class

In [ ]:
class DFA(Automaton[int, dict[str, int]]):
    next_state: int

    def __init__(self) -> None:
        self.transitions = {}
        self.start = 0
        self.states = { self.start }
        self.final = set()
        self.next_state = 1

    def add_state(self, transitions: dict[str, int]) -> None:
        self.states.add(self.next_state)
        self.transitions[self.next_state] = transitions
        self.next_state += 1

    def evaluate(self, s: str) -> bool:
        curr = self.start
        for c in s:
            curr = self.transitions[curr][c]
        return curr in self.final

In this implementation, the `add_state` function creates a new state using the
next unused integer as the new state ID.

## Equivalence and Emptiness <!-- TODO: rename this -->
Consider two DFAs, $D_1$ and $D_2$. Now suppose that the languages recognised
by the DFAs is given by $L_1$ and $L_2$ respectively.

If the DFAs are equivalent, then every string in $L_1$ must be in $L_2$. That is,
the automata agree on every string.

However, if they are not equivalent, then there exists at least one string which
is recognised by one DFA and not the other.

Thinking about this in set notation (remember that languages are just sets), 
we have that if $D_1$ and $D_2$ are not equivalent, then there is a string
$x$ such that $x \in L_1 \cap L_2^c$ or $x \in L_1^c \cap L_2$.

Hence, two DFAs are equivalent if the symmetric difference $$(L_1 \cap L_2^2) \cup (L_1^c \cap L_2)$$
of the languages they recognise is empty; i.e. there is no string which is in the language
of one DFA and not the other.

## Boolean operations with automata

### Union
Given two DFA, we want to be able to construct a new DFA which recognises
the language of strings which is recognised by either of the two input DFAs.
The standard way to do this is via a "product construction".

The idea behind a product construction is to create a new DFA which essentially
runs both DFA at once on the same string, and then checks if either is in a final
state at the end of the run.

To do this, we need our new DFA to keep track of the state each DFA is in at each
step. We can achieve this by storing the pair of the two states the two DFAs are in
as the current state of our new DFA.

If the states of the two DFAs are given by $Q_1$ and $Q_2$, then
our new DFA should have states $Q_1 \times Q_2$.

Then, since we want to accept any string which is accepted by any of the two DFAs,
any state pair which contains at least one final state in the original DFAs
should be marked as final.

[DIAGRAM HERE]

In code, this would be

In [ ]:
class DFA(DFA):
    def union(self, dfa: DFA) -> DFA:
        res = DFA()

        state_pairs = list(product(self.states, dfa.states))

        res.start = state_pairs.index((self.start, dfa.start))
        res.states = set(range(len(state_pairs)))
        res.final = set(i for i in res.states if state_pairs[i][0] in self.final or state_pairs[i][1] in dfa.final)
        res.next_state = len(state_pairs)
        res.transitions = {state: dict() for state in res.states}

        for state in res.states:
            for a in ALPHABET:
                res.transitions[state][a] = state_pairs.index((
                    self.transitions[state_pairs[state][0]][a],
                    dfa.transitions[state_pairs[state][1]][a]
                ))

        return res

### Intersection
To find a DFA recognising the intersection of the languages of two DFA, we only
need to slightly modify the algorithm for the union construction.

In particular, after running the DFA with state pairs,  <!-- TODO: reword -->
we only need to change the choice of final states.

For the intersection case, we only want to accept strings which are accepted
by both DFAs, so the final states should be the states where each of the states
in the pair are final states in the original DFAs.

In python, this would be 

In [ ]:
class DFA(DFA):
    def union(self, dfa: DFA) -> DFA:
        res = DFA()

        state_pairs = list(product(self.states, dfa.states))

        res.start = state_pairs.index((self.start, dfa.start))
        res.states = set(range(len(state_pairs)))
        # Notice the 'or' has become an 'and' on this line
        res.final = set(i for i in res.states if state_pairs[i][0] in self.final and state_pairs[i][1] in dfa.final)
        res.next_state = len(state_pairs)
        res.transitions = {state: dict() for state in res.states}

        for state in res.states:
            for a in ALPHABET:
                res.transitions[state][a] = state_pairs.index((
                    self.transitions[state_pairs[state][0]][a],
                    dfa.transitions[state_pairs[state][1]][a]
                ))

        return res

### Complement
Now we want to construct a DFA recognising the complement language of some
given DFA. That is, a DFA which accepts every string that is rejected by the
original DFA.

Since we just want to switch the evaluation of the DFA on every input, it suffices
to switch the final and non-final states.

In python, this would look like

In [ ]:
class DFA(DFA):
    def complement(self) -> DFA:
        dfa = deepcopy(self)
        dfa.final = self.states - self.final
        return dfa

### Symmetric Difference
Recall that the symmetric difference of two languages $L_1$ and $L_2$ is given
by $$(L_1 \cap L_2^c) \cup (L_1^c \cap L_2)$$. Since we already know how to construct
DFAs for the union, intersection and complement operations, we can compute
the symmetric difference by combining the constructions together.

In [ ]:
class DFA(DFA):
    def symmetric_difference(self, dfa: DFA) -> DFA
        self_not_d = self.intersection(dfa.complement())
        not_self_d = self.complement().intersection(dfa)

        return self_not_d.union(not_self_d)

## Equivalence and Emptiness
Now, given two DFA we can construct a new DFA which recognises all of the strings
on which the two original DFA disagree. It suffices to determine whether or not
this new DFA actually accepts any strings. If it doesn't recognise any strings, then
then two original DFA recognise the same language.

Since all we want is to find a single string which the DFA accepts, we just need to
find a path from the start state which leads to a final state, and then the DFA 
will accept the string formed by adding one character for each transition along 
this path, starting from the start state.

[DIAGRAM]

We will do so with a BFS traversal.

In [ ]:
class DFA(DFA):
    def is_empty(self) -> bool:
        seen = set()
        queue: deque[int] = deque([self.start])

        while len(queue) > 0:
            curr = queue.popleft()
            if curr in self.final:
                return False
            if curr in seen:
                continue
            seen.add(curr)

            for a in ALPHABET:
                state = self.transitions[curr][a]
                queue.append(state)

        return True

Finally, we can put everything together to obtain an equivalence algorithm for DFAs!

In [ ]:
class DFA(DFA):
    def is_equivalent(self, dfa: DFA) -> bool:
        return self.symmetric_difference(dfa).is_empty()

# Generalising to NFAs
Whilst DFAs are generally well behaved and have nice closure properties, as we saw
in the boolean operations section, often it is easier to represent languages with
more general objects. One such object is a nondeterministic finite automata, which
is a generalisation of DFA where states may have more than one outgoing transition for each
character, and a path to the final state is chosen nondeterministically.

In [ ]:
class NFA(Automaton[int, dict[str, set[int]]]):
    next_state: int

    def __init__(self) -> None:
        self.transitions = {}
        self.start = 0
        self.states = { self.start }
        self.final = set()
        self.next_state = 1

    def add_state(self, transitions: dict[str, set[int]]) -> None:
        self.states.add(self.next_state)
        self.transitions[self.next_state] = transitions
        self.next_state += 1
        
    def evaluate(self, s: str) -> bool:
        curr = { self.start }
        for c in s:
            curr = set().union(
                *(self.transitions[state][c] for state in curr)
            )
        return len(curr & self.final) > 0

[diagram of an NFA]

Consider the NFA shown above. If it is run on the string 'abba', [say what states it is in]

Notice that we have just applied a deterministic algorithm to determine whether
or not an NFA accepts a given string: we keep track at each step of which states
the NFA could possibly be in, and then accept the string if one of the possible
ending states is final in the original NFA.

We can create a DFA to complete this process in a similar manner to the product construction
for union and intersection, but now we need to keep track of a set of possible
states which might not be of a constant length at all times.

To achieve this, we can create a new DFA with states labelled by the powerset
of the set of states of the original DFA. Recall that the powerset of a set
is the set of all subsets, so the powerset of $\{1, 2, 3\}$ would be
$$\{\{\}, \{1\}, \{2\}, \{3\}, \{1,2\}, \{1,3\}, \{2,3\}, \{1,2,3\}\}.$$

Since we want the current state of this new DFA to store the set of states that
the NFA could be in, the choice of transitions follows naturally: for each
character $a$, the $a$-transition from each state should be the set of states
reachable from any state in the current set along a single $a$-transition in
the original NFA.

This way, the current state will always represent the set of possible states 
the NFA could be upon reading the current character.

Finally, the final states in the new DFA should be the sets of states which
contain at least one final state in the NFA, since this would correspond to
the possiblilty of ending in a final state in the original NFA.

We can program this so called 'determinisation' of the NFA as follows.

In [ ]:
class NFA(NFA):
    def determinise(self) -> DFA:
        def powerset(s: set[T]) -> list[tuple[T, ...]]:
            return list(chain.from_iterable(set(combinations(s, r)) for r in range(len(s)+1)))

        dfa = DFA()

        subsets = powerset(self.states)

        dfa.start = subsets.index((self.start,))
        dfa.states = set(range(len(subsets)))
        dfa.final = set(i for i in dfa.states if len(set(subsets[i]) & self.final) > 0)
        dfa.transitions = {state: dict() for state in dfa.states}
        dfa.next_state = len(subsets)

        for state, subset in enumerate(subsets):
            for a in ALPHABET:
                result = set().union(*(self.transitions[s].get(a, set()) for s in subset))
                dfa.transitions[state][a] = subsets.index(
                    tuple(sorted(result))
                )

        return dfa

However, there is a problem with this algorithm. If you look carefully,
you will notice that we never handled $\varepsilon$-transitions. If an NFA contains
$\varepsilon$-transitions, then after taking an $a$-transition it might be able to
reach extra states that aren't included in the target state in the DFA!

To fix this issue, we can first remove all $\varepsilon$-transitions from the
NFA before running the determinisation algorithm.

## Removing $\varepsilon$ States
aaaaa